In [1]:
import os
import ee
import geemap

# 设置HTTP和HTTPS代理
os.environ['HTTP_PROXY'] = 'http://127.0.0.1:7890'
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7890'

# 初始化Google Earth Engine和geemap
ee.Initialize()
Map = geemap.Map()

In [2]:
# 加载全球行政单元图层，选取南京市的行政边界
nanjing_boundary = ee.FeatureCollection('FAO/GAUL_SIMPLIFIED_500m/2015/level2') \
                    .filter(ee.Filter.eq('ADM2_NAME', 'Nanjing'))

# 定义感兴趣区域（ROI）为南京市的地理边界
roi = nanjing_boundary.geometry()

In [3]:
# 加载Sentinel-2图像集合，针对ROI进行筛选
sentinel_images = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
                    .filterBounds(roi) \
                    .filterDate('2023-01-01', '2023-12-31') \
                    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))

# 使用中值合成减少云影响，并将结果裁剪到ROI
composite = sentinel_images.median().clip(roi)

# 缩放图像值到0-1，以便于可视化
def scale_image(image):
    """将图像的像素值缩放到0-1范围内。"""
    return image.divide(10000).clamp(0, 1)

# 应用缩放函数到合成图像
scaled_composite = scale_image(composite)


In [4]:
# 计算增强植被指数（EVI）
def calculate_evi(image):
    """计算给定图像的增强植被指数（EVI）。"""
    evi = image.expression(
        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',
        {
            'NIR': image.select('B8'),  # Sentinel-2的近红外波段
            'RED': image.select('B4'),  # 红色波段
            'BLUE': image.select('B2')  # 蓝色波段
        }
    ).rename('EVI')
    
    return evi

# 应用EVI计算并使用阈值筛选出绿地区域
evi = calculate_evi(scaled_composite)
green_space = evi.gte(0.35).selfMask()  # 使用EVI阈值>=0.35来定义绿地

# 向地图添加合成图像和绿地图层
Map.addLayer(scaled_composite, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 1}, 'Composite Image')
Map.addLayer(green_space, {'palette': ['00FF00']}, 'Green Spaces')  # 绿色显示绿地

# 定位地图中心至南京，并设置合适的缩放级别
Map.centerObject(roi, 11)

# 显示地图
Map


Map(center=[31.927264240020794, 118.84598993136751], controls=(WidgetControl(options=['position', 'transparent…

In [5]:
# 定义像素的空间分辨率，Sentinel-2图像的典型分辨率为10米
pixel_scale = 10  # 以米为单位

# 计算绿地面积
green_space_area = green_space.multiply(ee.Image.pixelArea()).reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=roi,
    scale=pixel_scale,
    maxPixels=1e9
).getInfo()['EVI']

# 计算ROI（南京市）的总面积
total_area = roi.area().getInfo()

# 将面积从平方米转换为平方公里
green_space_area_km2 = green_space_area / 1e6
total_area_km2 = total_area / 1e6

# 计算绿地面积占城市总面积的百分比
green_space_percentage = (green_space_area_km2 / total_area_km2) * 100

print(f"南京市绿地面积: {green_space_area_km2:.2f} 平方公里")
print(f"南京市总面积: {total_area_km2:.2f} 平方公里")
print(f"绿地面积占比: {green_space_percentage:.2f}%")

南京市绿地面积: 860.85 平方公里
南京市总面积: 6754.65 平方公里
绿地面积占比: 12.74%
